In [ ]:
import numpy as np
import sympy
sympy.init_printing(print_builtin=True)

Import the package.

In [ ]:
import qsymm

# Model class

`Model` is an efficient way to store symbolic Hamiltonians. It supports initialization through `LaTeX`-like interface. Internally it stores the data as a dictionary of `{key: array}` where `key` is a `sympy` symbolic expression and `array` is a `numpy.ndarray`. It supports arythmetic operations with numbers, `sympy` expressions, `numpy.ndarrays` and other `Model` objects, among other useful functions.

Define 3D Rashba-Hamiltonian using a string.

In [ ]:
ham = ("hbar^2 / (2 * m) * (k_x**2 + k_y**2 + k_z**2) * eye(2) +" +
        "alpha * sigma_x * k_x + alpha * sigma_y * k_y + alpha * sigma_z * k_z")

Internally `qsymm.sympify` is used to parse the input.

In [ ]:
qsymm.sympify(ham)

Define `Model` object.

In [ ]:
H = qsymm.Model(ham)

We can inspect the content dictionary.

In [ ]:
H

Transform `Model` to `sympy` representation.

In [ ]:
H.tosympy(nsimplify=True)

`Model` has a `momenta` attribute, specifying which symbols are considered the momentum variables.

In [ ]:
H.momenta

By default 3-dimensional space is assumed, to define 2-dimensional system with only `k_x` and `k_z` momentum variables, use:

In [ ]:
ham2D = ("hbar^2 / (2 * m) * (k_x**2 + k_z**2) * eye(2) +" +
        "alpha * sigma_x * k_x + alpha * sigma_y * k_z")
H2D = qsymm.Model(ham2D, momenta=['k_x', 'k_z'])
H2D.momenta

# PointGroupElement class

`PointGroupElement` is a class to store discrete symmetry operators. 

+ The first attribure `R` is the real space rotation action of the symmetry as an orthogonal matrix. As exact arithmetic is assumed, it must be provided either as an integer array or a `sympy` matrix.
+ The second and third attributes are booleans, specifying whether the operator is antiunitary (such as time reversal or particle-hole) and whether it is an antisymmetry (such as particle-hole or chiral) respectively.
+ The fourth attribute is optional, it defines the unitary action of the symmetry operator on the Hilbert-space. If not provided, it is treated unknown for symmetry finding purposes.

It supports group multiplication, inversion and can be applied to `Model` objects to carry out the transformation.

As an example, define cubic point group by its generators (the unitary action is left unspecified).

In [ ]:
# Identity
E = qsymm.identity(3)
# Inversion
I = qsymm.inversion(3)
# 4-fold rotation
C4 = qsymm.rotation(1/4, [1, 0, 0])
# 3-fold rotation
C3 = qsymm.rotation(1/3, [1, 1, 1])
# Time reversal
TR = qsymm.time_reversal(3)
# Particle-hole
PH = qsymm.particle_hole(3)

cubic_gens = {I, C4, C3, TR, PH}

`qsymm.groups` offers a variety of methods to interact with sets of `PointGroupElement` objects. Here we use `generate_group` to generate the full group from the generators.

In [ ]:
cubic_group = qsymm.groups.generate_group(cubic_gens)
print(len(cubic_group))

Can also get the cubic group directly.

In [ ]:
cubic_group = qsymm.groups.cubic()

PointGroupElements can be multiplied, inverted and be applied to Models

In [ ]:
C3 * C4

In [ ]:
C3**-1

In [ ]:
TR.apply(H).tosympy(nsimplify=True)

# ContinuousGroupGenerator class

`ContinuousGroupGenerator` is a class to store continuous symmetry generators, such as local conserved quantities or continuous spatial rotations.

+ The first attribure `R` is the real space rotation generator of the symmetry as an antisymmetric imaginary matrix. Default is zero.
+ The second attribute the generator of the unitary action of the symmetry operator on the Hilbert-space as a Hermitian matrix. Default is zero.

As an example, let us define spin-z conservation:

In [ ]:
sz = qsymm.ContinuousGroupGenerator(None, np.array([[1, 0], [0, -1]]))
sz

Applying it to a `Model` calculates the commutator. Spin-z is not conserved in `H`, as the commutator is nonzero.

In [ ]:
sz.apply(H).tosympy(nsimplify=True)

# Symmetry finder

The symmetry finding routines can be accessed through `qsymm.symmetries`. For example let us find out whether the 3D Rashba Hamiltonian defined earlier has cubic group symmetry.

In [ ]:
discrete_symm, continuous_symm = qsymm.symmetries(H, cubic_group)
print(len(discrete_symm), len(continuous_symm))

It has 48 discrete symmetries (cubic group without inversion and time-reversal) and no conserved quantities.

For more detailed examples see [`symmetry_finder.ipynb`](symmetry_finder.ipynb) and [`kekule.ipynb`](kekule.ipynb).

In [ ]:
discrete_symm

# Hamiltonian generator

The hamiltonian generator algorithms can be accessed through `qsymm.continuum_hamiltonian` and `qsymm.bloch_family`.

For example let us generate all 2-band k.p Hamiltonians with the previously found group.

In [ ]:
family = qsymm.continuum_hamiltonian(discrete_symm, dim=3, total_power=2, prettify=True)
qsymm.display_family(family)

It is exactly the Hamiltonian family we started with.

For more detailed examples see [`kdotp_generator.ipynb`](kdotp_generator.ipynb), [`bloch_generator.ipynb`](bloch_generator.ipynb) and [`kekule.ipynb`](kekule.ipynb).